In [2]:
# 필요한 라이브러리 임포트
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.nn import GATConv
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# CUDA 디바이스 어설션 활성화
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [4]:
# 데이터 로드 함수 정의
def load_data(files):
    df_list = []
    for file in files:
        df = pd.read_csv(file)
        df.drop(columns=['time'], inplace=True)  # 'time' 열 제거
        df_list.append(df)
    combined_df = pd.concat(df_list, axis=0, ignore_index=True)
    return combined_df

# 데이터 준비 함수 정의
def prepare_data(df, label_column='attack'):
    inputs = df.drop(columns=[label_column]).values.astype(float)
    labels = df[label_column].values.astype(float)
    return torch.tensor(inputs, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)

In [5]:
# 파일 경로 설정
train_files = [
    r'C:\Users\Researcher\Desktop\MTAD-GAT\train\train1.csv',
    r'C:\Users\Researcher\Desktop\MTAD-GAT\train\train2.csv'
]
test_files = [
    r'C:\Users\Researcher\Desktop\MTAD-GAT\test1\test1.csv',
    r'C:\Users\Researcher\Desktop\MTAD-GAT\test1\test2.csv'
]

# 데이터 로드
train_df = load_data(train_files)
test_df = load_data(test_files)

# 데이터 전처리
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_df.drop(columns=['attack']))
test_scaled = scaler.transform(test_df.drop(columns=['attack']))

train_scaled_df = pd.DataFrame(train_scaled, columns=train_df.columns.drop('attack'))
test_scaled_df = pd.DataFrame(test_scaled, columns=test_df.columns.drop('attack'))

train_scaled_df['attack'] = train_df['attack'].values
test_scaled_df['attack'] = test_df['attack'].values

train_inputs, train_labels = prepare_data(train_scaled_df, label_column='attack')
test_inputs, test_labels = prepare_data(test_scaled_df, label_column='attack')

train_dataset = torch.utils.data.TensorDataset(train_inputs, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_inputs, test_labels)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# 데이터 유효성 검사
for inputs, labels in train_loader:
    print(f"Batch size: {inputs.size(0)}")
    print(f"Inputs shape: {inputs.shape}")
    print(f"Labels shape: {labels.shape}")
    assert not torch.any(torch.isnan(inputs)), "NaNs found in inputs"
    assert not torch.any(torch.isnan(labels)), "NaNs found in labels"
    assert not torch.any(torch.isinf(inputs)), "Infs found in inputs"
    assert not torch.any(torch.isinf(labels)), "Infs found in labels"
    break  # 한 배치만 확인

# 상관 행렬 계산
corr_matrix = train_scaled_df.drop(columns=['attack']).corr().values

# create_edge_index 함수 정의
def create_edge_index(corr_matrix, threshold):
    edge_index = []
    num_nodes = corr_matrix.shape[0]
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j and abs(corr_matrix[i, j]) > threshold:
                edge_index.append([i, j])
    return torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# edge_index 생성
num_nodes = train_scaled_df.drop(columns=['attack']).shape[1]
edge_index = create_edge_index(corr_matrix, threshold=0.5)
print(f'Edge index shape: {edge_index.shape}')
print(f'Edge index max value: {edge_index.max()}')

# edge_index가 num_nodes보다 작은지 확인
assert edge_index.max().item() < num_nodes, f"edge_index의 최대값이 num_nodes보다 큽니다. edge_index max: {edge_index.max().item()}, num_nodes: {num_nodes}"

Batch size: 32
Inputs shape: torch.Size([32, 62])
Labels shape: torch.Size([32])
Edge index shape: torch.Size([2, 348])
Edge index max value: 58


In [6]:
# MTAD-GAT 모델 정의
class MTADGAT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads):
        super(MTADGAT, self).__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=num_heads, concat=True)
        self.gat2 = GATConv(hidden_dim * num_heads, output_dim, heads=1, concat=True)
        self.gru = nn.GRU(output_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = torch.relu(x)
        x = self.gat2(x, edge_index)
        x = x.view(1, -1, x.size(1))  # GRU를 위한 리쉐이프
        x, _ = self.gru(x)
        x = self.fc(x.view(-1, x.size(2)))
        return x

# 모델 초기화 및 학습 설정
input_dim = train_scaled_df.drop(columns=['attack']).shape[1]
hidden_dim = 64
output_dim = 32
num_heads = 8

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MTADGAT(input_dim, hidden_dim, output_dim, num_heads).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델을 GPU로 이동하기 전에 검증
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

gat1.att_src tensor([[[ 0.1968, -0.1900,  0.2299,  0.0657,  0.1033,  0.1641, -0.2338,
           0.0103, -0.2840,  0.1394,  0.0403, -0.1741,  0.0053, -0.2253,
          -0.0931, -0.1861,  0.1950,  0.0024, -0.1879,  0.1865,  0.2577,
          -0.0346,  0.2382,  0.1233,  0.1066, -0.1739,  0.1632,  0.2557,
           0.0287, -0.2078,  0.1023, -0.0650, -0.1042, -0.1612, -0.0004,
          -0.1939, -0.2456,  0.2209,  0.2489,  0.0238, -0.2120, -0.1473,
          -0.0364, -0.0971,  0.1044, -0.1600, -0.2745, -0.2816, -0.2504,
           0.0728,  0.2660,  0.0652,  0.2020,  0.1638,  0.1892,  0.0936,
          -0.1197,  0.2843, -0.1467,  0.1198,  0.0875,  0.1369, -0.1244,
           0.0118],
         [ 0.0972, -0.1494,  0.2737,  0.0257, -0.2754, -0.1604, -0.0216,
           0.1121, -0.1644,  0.0285, -0.2469, -0.2636, -0.0228,  0.2414,
          -0.2781, -0.2551,  0.0134, -0.2237, -0.1637,  0.1804, -0.2187,
           0.1227, -0.1023, -0.1773, -0.1496, -0.1562, -0.2327, -0.2743,
           0.0572,

In [7]:
# 상관 행렬 기반 edge_index 생성 함수 정의
def create_edge_index(corr_matrix, threshold=0.5):
    edge_index = []
    num_nodes = corr_matrix.shape[0]
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j and abs(corr_matrix[i, j]) >= threshold:
                edge_index.append([i, j])

    edge_index = np.array(edge_index)

    # 노드 수 초과하는 인덱스 제거
    valid_indices = (edge_index[:, 0] < num_nodes) & (edge_index[:, 1] < num_nodes)
    edge_index = edge_index[valid_indices]

    return torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# 상관 행렬 계산
corr_matrix = train_scaled_df.drop(columns=['attack']).corr().values

# edge_index 생성
num_nodes = train_scaled_df.drop(columns=['attack']).shape[1]
edge_index = create_edge_index(corr_matrix, threshold=0.5)
print(f'Edge index shape: {edge_index.shape}')
print(f'Edge index max value: {edge_index.max()}')

# edge_index가 num_nodes보다 작은지 확인
assert edge_index.max() < num_nodes, f"edge_index의 최대값이 num_nodes보다 큽니다. edge_index max: {edge_index.max()}, num_nodes: {num_nodes}"

Edge index shape: torch.Size([2, 348])
Edge index max value: 58


In [11]:
# 모델 학습 및 평가
epochs = 50
train_losses = []
test_losses = []
precisions = []
recalls = []

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # 입력 크기와 값 확인
        assert inputs.size(0) == labels.size(0), f"Batch size mismatch: {inputs.size(0)} vs {labels.size(0)}"
        assert not torch.any(torch.isnan(inputs)), "NaNs found in inputs"
        assert not torch.any(torch.isnan(labels)), "NaNs found in labels"

        # edge_index 필터링
        mask = edge_index < inputs.size(0)
        edge_index_filtered = edge_index[:, mask[0] & mask[1]]

        # edge_index 확인
        assert edge_index_filtered.max().item() < inputs.size(0), f"edge_index의 최대값이 inputs의 노드 수 {inputs.size(0)}보다 큽니다. edge_index max: {edge_index_filtered.max().item()}, input nodes: {inputs.size(0)}"

        optimizer.zero_grad()
        outputs = model(inputs, edge_index_filtered.to(device))
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    train_losses.append(total_loss / len(train_loader))
    # 테스트 손실 및 성능 평가
    model.eval()
    all_preds = []
    all_labels = []
    total_test_loss = 0
    predictions = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs, edge_index_filtered.to(device))
            loss = criterion(outputs.squeeze(), labels)
            total_test_loss += loss.item()
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            predictions.append(outputs.cpu().numpy())
    test_losses.append(total_test_loss / len(test_loader))
    
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    precisions.append(precision)
    recalls.append(recall)
    
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {total_loss / len(train_loader):.4f}, Test Loss: {total_test_loss / len(test_loader):.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    
    # 모델 체크포인트 저장
    torch.save(model.state_dict(), f'model_checkpoint_epoch_{epoch + 1}.pth')


C:\Users\Researcher\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/50, Train Loss: 0.0009, Test Loss: 0.0221, Precision: 0.4803, Recall: 0.5000


C:\Users\Researcher\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/50, Train Loss: 0.0009, Test Loss: 0.0100, Precision: 0.4803, Recall: 0.5000


C:\Users\Researcher\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch 48/50, Train Loss: 0.0009, Test Loss: 0.0208, Precision: 0.4803, Recall: 0.5000


C:\Users\Researcher\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 49/50, Train Loss: 0.0009, Test Loss: 0.0098, Precision: 0.4803, Recall: 0.5000


C:\Users\Researcher\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 50/50, Train Loss: 0.0008, Test Loss: 0.0080, Precision: 0.4803, Recall: 0.5000


In [12]:
# 평가 지표 계산
predictions = np.concatenate(predictions).flatten()
mse = np.mean((test_labels.numpy() - predictions) ** 2)
mae = mean_absolute_error(test_labels.numpy(), predictions)
r2 = r2_score(test_labels.numpy(), predictions)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 0.007998411543667316
Mean Absolute Error: 0.021648818626999855
R^2 Score: 0.7887809885328958


In [13]:
# 이상 탐지
anomaly_threshold = mse + 3 * np.std(predictions - test_labels.numpy().reshape(-1, 1))

anomalies = (np.abs(predictions - test_labels.numpy().reshape(-1, 1)) > anomaly_threshold).astype(int)

# Classification metrics
accuracy = accuracy_score(test_labels.numpy(), anomalies)
f1 = f1_score(test_labels.numpy(), anomalies, average='macro')
cm = confusion_matrix(test_labels.numpy(), anomalies)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(cm)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

# 예측 및 이상치 시각화
plt.figure(figsize=(15, 5))
plt.plot(test_labels.numpy(), label='Actual', color='blue')
plt.plot(predictions, label='Predicted', color='green')
plt.scatter(np.where(anomalies == 1)[0], predictions[anomalies == 1], color='red', label='Anomalies')
plt.legend()
plt.show()

# 학습 곡선 시각화
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Testing Loss')
plt.show()

# Precision and Recall 시각화
plt.figure(figsize=(10, 5))
plt.plot(precisions, label='Precision')
plt.plot(recalls, label='Recall')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.legend()
plt.title('Precision and Recall')
plt.show()

MemoryError: Unable to allocate 736. GiB for an array with shape (444600, 444600) and data type float32